In [1]:
from py_files import preprocessing as pp
from py_files import Augmentation as ag
from py_files import pic_class as pc
from py_files import TrainTestSplit as ts
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib agg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

Using TensorFlow backend.


In [2]:
source = os.getcwd()
source

'/Users/juliawang/Desktop/Direct/Spicy-Raman/model_train_and_validate'

## DATA IMPORTING

In [3]:
train_dir = os.path.join(source, 'trainingdata_Pictures')
validation_dir = os.path.join(source, 'validationdata_Pictures')

## DATA INFORMATION

In [4]:
train_fluorescent_dir = os.path.join(train_dir, 'fluorescent_Pictures')
train_nylon_dir = os.path.join(train_dir, 'nylon_Pictures')
train_polyamide_dir = os.path.join(train_dir, 'polyamide_Pictures')
train_polyethylene_dir = os.path.join(train_dir, 'polyethylene_Pictures')
train_others_dir = os.path.join(train_dir, 'others_Pictures')

validation_fluorescent_dir = os.path.join(validation_dir, 'fluorescent_Pictures')
validation_nylon_dir = os.path.join(validation_dir, 'nylon_Pictures')
validation_polyamide_dir = os.path.join(validation_dir, 'polyamide_Pictures')
validation_polyethylene_dir = os.path.join(validation_dir, 'polyethylene_Pictures')
validation_others_dir = os.path.join(validation_dir, 'others_Pictures')

num_fluorescent_tr = len(os.listdir(train_fluorescent_dir))
num_nylon_tr = len(os.listdir(train_nylon_dir))
num_polyamide_tr = len(os.listdir(train_polyamide_dir))
num_polyethylene_tr = len(os.listdir(train_polyethylene_dir))
num_others_tr = len(os.listdir(train_others_dir))

num_fluorescent_val = len(os.listdir(validation_fluorescent_dir))
num_nylon_val = len(os.listdir(validation_nylon_dir))
num_polyamide_val = len(os.listdir(validation_polyamide_dir))
num_polyethylene_val = len(os.listdir(validation_polyethylene_dir))
num_others_val = len(os.listdir(validation_others_dir))

total_train = num_fluorescent_tr + num_polyamide_tr + num_nylon_tr + num_polyethylene_tr + num_others_tr
total_val = num_fluorescent_val + num_polyamide_val + num_nylon_val + num_polyethylene_val + num_others_val

print("Total training images:", total_train)
print("Total validation images:", total_val)

Total training images: 463
Total validation images: 122


## DATA TRAINING

In [7]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [8]:
train_image_generator = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [9]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 463 images belonging to 5 classes.


In [10]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 122 images belonging to 5 classes.


In [11]:
sample_training_images, _ = next(train_data_gen)

In [12]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [22]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=122 // batch_size
)

Epoch 1/15
2/3 [===================>..........] - ETA: 4s - loss: -1302.3045 - accuracy: 0.1691

ValueError: Empty training data.

In [27]:
model.save("Spicy_Raman_Saved_Model.h5")

## IMAGE PREDICTION

In [23]:
import os
 
import numpy as np
 
import matplotlib.pyplot as plt
 
import tensorflow as tf
 
from keras.models import load_model
from keras.preprocessing import image
 

In [25]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [28]:
loaded_model = tf.keras.models.load_model('Spicy_Raman_Saved_Model.h5')
loaded_model.layers[0].input_shape #(None, 160, 160, 3)

(None, 150, 150, 3)

In [31]:
IMG_SIZE =150

In [32]:
batch_holder = np.zeros((20, IMG_SIZE, IMG_SIZE, 3))
img_dir= '/Users/juliawang/Desktop/Direct/Spicy-Raman/model_train_and_validate/validationdata_Pictures/nylon_Pictures'
for i,img in enumerate(os.listdir(img_dir)):
  img = image.load_img(os.path.join(img_dir,img), target_size=(IMG_SIZE,IMG_SIZE))
  batch_holder[i, :] = img

In [34]:
result=loaded_model.predict_classes(batch_holder)
 
fig = plt.figure(figsize=(20, 20))
 
for i,img in enumerate(batch_holder):
  fig.add_subplot(4,5, i+1)
  #plt.title(get_label_name(result[i][0]))
  plt.imshow(img/256.)
  
plt.show()

/Users/juliawang/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # Remove the CWD from sys.path while we load stuff.
